## Learning objectives
* Students should be able to assess the structure and cleanliness of their dataset, including size and shape of data, number of variables of each type 
* Students should be able to describe their findings, translate results from code to text using Markdown comments in the Jupyter Notebook, and summarize their thought process in a narrative
* Students should be able to modify the raw data to prepare a clean data set -- including copying data, removing or replacing missing and incoherent data, dropping columns, removing duplicates in Pandas and Jupyter -- and explain and justify their decisions in markdown in their Jupyter notebook
* Students should be able to assess whether their data is “Tidy” and identify appropriate steps and write and  execute code to arrange it into a tidy format - including merging, reshaping, subsetting, grouping, sorting, making appropriate new columns  - and explain and justify their decisions in markdown in their Jupyter notebook
* Students should be able to identify several relevant summary measures, illustrate data using appropriate plots, and explain and justify their decisions in markdown in their Jupyter notebook
* Student should assess the summaries and plots and appraise the need for repeated or further analysis, and justify decisions in markdown


## Assess the structure and cleanliness

* Take “header” of data, # columns/rows, length, shape, size
* Cheat sheet for commands
* make sure data is in right type - categorical, continuous numerical, string, datetime
* use markdown to comment


## Describe findings, translate results into Markdown text
 This is more of an overarching goal that should be woven in through the lesson

## Modify the dataset -- cleaning

* Make a copy of the data
in place vs. not
pandas.DataFrame.copy() - make a copy of data frame
* Assigning to new variable/df names 
* Dealing with missing data
Pros and cons of dropping NAs and inconsistent data
* Dealing with incoherent data (NA, na, N/A, n/a, ND, not done, XXXX) misspellings, etc
Regex, data transformation to address inconsistency (fill_na)
* Dropping columns
df.drop()
* Removing duplicates
df.drop_duplicates()


## Prepare the data structure -- tidy data

* Describe Tidy Data - each variable has it’s own column, each observation has its own row. 
* Reshaping (if necessary) 
renaming columns (from names that don’t make sense, names with strange characters, etc to names that make sense) 
* Merging datasets,
Pandas - merge()[left, right, inner, outer], concat()
* Subsetting data
Indexing - numerically(zero indexing) vs by names, boolean indices, 
Slicing - base python and/or pandas
List comprehensions
Regular expressions (complicated! But useful and important)
* Grouping data, Indexing DataFrames
df.groupby(by="col")
* Sorting data
df.sort_values()
df.sort_index()
* Creating new variables/columns (for transformation (log, sqrt, etc)


## Summarize and plot

Summaries (but can’t *say* statistics…)
* Sort data
* Can make note about using numpy functions, dif between dataframe and array
Good Plots for the data/variable type



Plots 
* of subsets, 
* single variables
* pairs of variables
* Matplotlib syntax (w/ seaborn for defaults (prettier, package also good for more analysis later...))

Exploring is often iterative - summarize, plot, summarize, plot, etc. - sometimes it branches…


## Interpret plots